# Total precipitation quantiles

In [2]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu
import seeps_utils as seeps

In [5]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [6]:
path_verif = conf['IMERG']['save_loc_verif']+'Histogram_vals_daily_IMERG.nc'

In [7]:
path_verif

'/glade/campaign/cisl/aiml/ksha/CREDIT_cp/VERIF/IMERG/Histogram_vals_daily_IMERG.nc'

## Verification setup

In [8]:
# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob('/glade/campaign/cisl/aiml/ksha/IMERG_V7/GPM_3B_V07/year_*_1deg_interp.zarr'))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0]-1, year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')

In [10]:
base_dir = '/glade/campaign/cisl/aiml/ksha/CREDIT_cp/'
ds_TP = xr.open_dataset(base_dir+'VERIF/fuxi_w_physics/Histogram_vals_005d_fuxi_physics.nc')

In [15]:
ds_ERA5_merge_sub = ds_ERA5_merge.sel(time=ds_TP['time'])

In [16]:
ds_ERA5_merge_sub

<xarray.Dataset>
Dimensions:              (latitude: 181, longitude: 360, time: 731)
Coordinates:
  * latitude             (latitude) int64 90 89 88 87 86 ... -86 -87 -88 -89 -90
  * longitude            (longitude) int64 0 1 2 3 4 5 ... 355 356 357 358 359
  * time                 (time) datetime64[ns] 2020-01-06 ... 2022-01-05
Data variables:
    total_precipitation  (time, latitude, longitude) float32 dask.array<chunksize=(1, 181, 360), meta=np.ndarray>

In [17]:
ds_ERA5_merge_sub.to_netcdf(path_verif)